In [364]:
import pandas as pd
import numpy as np

In [365]:
data = [[1, 1, '2023-01-15', 5, 10.0], [2, 2, '2023-01-20', 4, 15.0], [3, 3, '2023-03-10', 3, 18.0], [4, 4, '2023-04-05', 1, 20.0], [5, 1, '2023-05-20', 2, 10.0], [6, 2, '2023-06-12', 4, 15.0], [7, 5, '2023-06-15', 5, 12.0], [8, 3, '2023-07-24', 2, 18.0], [9, 4, '2023-08-01', 5, 20.0], [10, 5, '2023-09-03', 3, 12.0], [11, 1, '2023-09-25', 6, 10.0], [12, 2, '2023-11-10', 4, 15.0], [13, 3, '2023-12-05', 6, 18.0], [14, 4, '2023-12-22', 3, 20.0], [15, 5, '2024-02-14', 2, 12.0]]
sales = pd.DataFrame(data,columns=['sale_id', 'product_id', 'sale_date', 'quantity', 'price']).astype({'sale_id': 'int64', 'product_id': 'int64', 'sale_date': 'datetime64[ns]', 'quantity': 'int64', 'price': 'float64'})

data = [[1, 'Warm Jacket', 'Apparel'], [2, 'Designer Jeans', 'Apparel'], [3, 'Cutting Board', 'Kitchen'], [4, 'Smart Speaker', 'Tech'], [5, 'Yoga Mat', 'Fitness']]
products = pd.DataFrame(data,columns=['product_id', 'product_name', 'category']).astype({'product_id': 'int64', 'product_name': 'string', 'category': 'string'})


In [366]:
sales['season']= np.where(sales['sale_date'].apply(str).str.contains('-01-|-12-|-02-'),'Winter',
                          np.where(sales['sale_date'].apply(str).str.contains('-03-|-04-|-05-'),'Spring',
                          np.where(sales['sale_date'].apply(str).str.contains('-08-|-06-|-07-'),'Summer',
                                   np.where(sales['sale_date'].apply(str).str.contains('-09-|-10-|-11-'),'Fall',''))))

In [367]:
df = pd.merge(sales,products,on='product_id')
df

,sale_id,product_id,sale_date,quantity,price,season,product_name,category
0,1,1,2023-01-15,5,10.0,Winter,Warm Jacket,Apparel
1,2,2,2023-01-20,4,15.0,Winter,Designer Jeans,Apparel
2,3,3,2023-03-10,3,18.0,Spring,Cutting Board,Kitchen
3,4,4,2023-04-05,1,20.0,Spring,Smart Speaker,Tech
4,5,1,2023-05-20,2,10.0,Spring,Warm Jacket,Apparel
5,6,2,2023-06-12,4,15.0,Summer,Designer Jeans,Apparel
6,7,5,2023-06-15,5,12.0,Summer,Yoga Mat,Fitness
7,8,3,2023-07-24,2,18.0,Summer,Cutting Board,Kitchen
8,9,4,2023-08-01,5,20.0,Summer,Smart Speaker,Tech
9,10,5,2023-09-03,3,12.0,Fall,Yoga Mat,Fitness


In [368]:
df=df.groupby(['category','season','price'])[['quantity']].sum().reset_index()
#
df['revenue'] = df['quantity'] * df['price']
df=df.groupby(['category','season']).agg({'quantity':'sum','revenue':'sum'}).reset_index()
df['q_rank'] = df.groupby(['season'])['quantity'].rank(method='dense',ascending=False)
df['r_rank'] = df.groupby(['season'])['revenue'].rank(method='dense',ascending=False)
df.sort_values('q_rank')

,category,season,quantity,revenue,q_rank,r_rank
0,Apparel,Fall,10,120.0,1.0,1.0
3,Apparel,Winter,9,110.0,1.0,1.0
7,Kitchen,Spring,3,54.0,1.0,1.0
5,Fitness,Summer,5,60.0,1.0,2.0
11,Tech,Summer,5,100.0,1.0,1.0
2,Apparel,Summer,4,60.0,2.0,2.0
4,Fitness,Fall,3,36.0,2.0,2.0
1,Apparel,Spring,2,20.0,2.0,2.0
9,Kitchen,Winter,6,108.0,2.0,2.0
12,Tech,Winter,3,60.0,3.0,3.0


In [372]:
results = df[(df['q_rank']==1) & (df['r_rank']==1)][['season','category','quantity','revenue']].sort_values(by='season')
results

,season,category,quantity,revenue
0,Fall,Apparel,10,120.0
7,Spring,Kitchen,3,54.0
11,Summer,Tech,5,100.0
3,Winter,Apparel,9,110.0
